In [5]:
import sys; sys.path.append('../..')
from src.results import read_results

trades = read_results.get_trades("solarsail")
print(len(list(trades)))

654


In [6]:
import datetime
import collections

rois_by_time = {}

profit = sum(t.get_profit_loss() for t in trades)
print(collections.Counter(t.is_win() for t in trades), profit)

for t in trades:
    key = t.get_start().time()
    rois_by_time[key] = rois_by_time.get(key, []) + [t.get_profit_loss()]

def group_rois_by_time(rois_by_time):
    grouped_rois = {}
    for time, rois in rois_by_time.items():
        new_time = datetime.time(time.hour, time.minute // 15 * 15)
        grouped_rois[new_time] = grouped_rois.get(new_time, []) + rois
    return grouped_rois

import plotly.express as px
import pandas as pd

data= pd.DataFrame([
    {
        'time': time,
        'freq': len(rois),
        'roi': sum(rois) / len(rois),
    }
    for time, rois in sorted(group_rois_by_time(rois_by_time).items())
])



Counter({False: 383, True: 271}) 156.62550000000041


In [12]:
px.bar(data, x="time", y="freq", title=f"Trades by Entry Time").show()
px.bar(data, x="time", y="roi", title=f"Profit by Entry Time").show()